In [81]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unicodedata
# import texthero as hero
from nltk.corpus import stopwords
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope

# Carregando e limpando os dados

In [82]:
pagamentos = pd.read_csv('pagamentos.csv')
pagamentos.info()
# pagamentos.value_counts()
print(pagamentos.nunique())
pagamentos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427873 entries, 0 to 427872
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   data_publicacao          427873 non-null  object
 1   fase                     427873 non-null  object
 2   credor                   427873 non-null  object
 3   valor                    427873 non-null  object
 4   numero                   427873 non-null  object
 5   cpf_ou_cnpj              427873 non-null  object
 6   data_pagamento           427873 non-null  object
 7   numero_processo          253290 non-null  object
 8   bem_ou_servico_prestado  420978 non-null  object
 9   natureza                 422333 non-null  object
 10  acao                     407378 non-null  object
 11  funcao                   407414 non-null  object
 12  subfuncao                407405 non-null  object
 13  processo_licitatorio     407396 non-null  object
 14  fonte_recurso       

,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
0,01/01/2010,EMPENHO,METALPIRES INDUSTRIA E COM LT,"R$ 203.690,00",10000348,06.965.538/0001-00,01/01/2010,157/2009,REF AQUISICAO DE MOBILIARIOS E EQUIPAMENTOS PA...,449052090000 - Out.Equip. e Mat. Permanente,2040 - Equipar e Reequipar Unidades Escolares ...,12 - EDUCACAO,361 - ENSINO FUNDAMENTAL,PREGAO,0001 - REC.IMP.TRANSF.EDUCACAO 25%
1,01/01/2010,EMPENHO,INST.NAC.DE SEGURIDADE SOCIAL,"R$ 1.500.000,00",10000195,01.404.357/0001-51,01/01/2010,NaN,REFERENTE A DESPESA COM PARCE- LAMENTO DA DI...,469071030000 - INSS/PARCELAMENTO,2145 - Administracao de encagos especiais,28 - ENCARGOS ESPECIAIS,843 - SERVICO DA DIVIDA INTERNA,ISENTO,0000 - RECURSOS ORDINARIOS
2,01/01/2010,EMPENHO,GASAUTO LOCAC?O DE VEICULOS L,"R$ 18.816,00",113/2009,00.426.288/0001-58,01/01/2010,039/2009,locac?o de 04 motocicletas 150cc com com parti...,339039780000 - locacao de veiculos,2141 - Seguranca no transito,26 - TRANSPORTE,452 - SERVICOS URBANOS,CONVITE,0050 - REC.PROPRIAS ENT.ADM.
3,02/01/2010,EMPENHO,SALUSTIANO JOSE ALVES DIAS,"R$ 4.800,00",10000024,407.981.865-34,02/01/2010,230/2007,"TERMO DE RENOVACAO DO ADITIVO 5025271/2009, R...",339039780000 - locacao de veiculos,2097 - Pavimentacao e infra-estrutura,15 - URBANISMO,451 - INFRA-ESTRUTURA URBANA,TOMADA DE PRECO,0000 - RECURSOS ORDINARIOS
4,02/01/2010,EMPENHO,VALDEANE DOS SANTOS DIAS,"R$ 3.600,00",10000494,006.534.185-66,02/01/2010,NaN,REF. ALUGUEL DO IMOVEL ONDE FUNCIONA O NUCL...,339039100000 - Locacao de imoveis,2084 - Programa Errad.Trabalho Infantil PETI,08 - ASSISTENCIA SOCIAL,243 - ASSISTENCIA A CRIANCA E AO ADO,ISENTO,0000 - RECURSOS ORDINARIOS


In [83]:
# Removendo duplicatas
pagamentos = pagamentos[pagamentos['fase'] == 'PAGAMENTO']
pagamentos.drop(['data_publicacao', 'fase', 'numero', 
                'cpf_ou_cnpj', 'data_pagamento', 'numero_processo'],
                inplace=True, axis=1)
print(pagamentos.info())
print(pagamentos.nunique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181527 entries, 13 to 427868
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   credor                   181527 non-null  object
 1   valor                    181527 non-null  object
 2   bem_ou_servico_prestado  179835 non-null  object
 3   natureza                 175987 non-null  object
 4   acao                     161047 non-null  object
 5   funcao                   161068 non-null  object
 6   subfuncao                161059 non-null  object
 7   processo_licitatorio     161057 non-null  object
 8   fonte_recurso            161059 non-null  object
dtypes: object(9)
memory usage: 13.8+ MB
None
credor                      7505
valor                      83054
bem_ou_servico_prestado    93641
natureza                     556
acao                         526
funcao                        36
subfuncao                     61
processo_licitator

In [84]:
num_entradas_repetidas = len(pagamentos)
pagamentos = pagamentos.drop_duplicates()
num_entradas_unicas = len(pagamentos)
print(f'Numero de entradas duplicadas: {num_entradas_repetidas - num_entradas_unicas}\nNumero de entradas unicas: {num_entradas_unicas}')

Numero de entradas duplicadas: 21316
Numero de entradas unicas: 160211


In [85]:
# Limpando float
def currency_to_float(value):
    """Converte de R$ 69.848,70 (str) para 69848.70 (float)."""
    try:
        cleaned_value = value.replace("R$", "").replace(".", "").replace(",", ".")
        return float(cleaned_value)
    except ValueError:
        return

pagamentos['valor'] = pagamentos['valor'].apply(currency_to_float)
pagamentos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160211 entries, 13 to 427803
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   credor                   160211 non-null  object 
 1   valor                    160211 non-null  float64
 2   bem_ou_servico_prestado  158900 non-null  object 
 3   natureza                 155488 non-null  object 
 4   acao                     141462 non-null  object 
 5   funcao                   141482 non-null  object 
 6   subfuncao                141474 non-null  object 
 7   processo_licitatorio     141472 non-null  object 
 8   fonte_recurso            141474 non-null  object 
dtypes: float64(1), object(8)
memory usage: 12.2+ MB


In [86]:
# Limpando processo_licitatorio

def limpar_proc_licitatorio(proc):
    isento = ['ISENTO', 'ENTO', 'Isento', 'SENTO', 'sento']
    pregao = ['PREGAO', 'REGAO', 'Pregao', 'EGAO', 'egao']
    dispensa = ['DISPENSA', 'SPENSA', 'ISPENSA', 'Dispensa', 'PENSA', 'pensa']
    concorrencia = ['CONCORENCIA', 'ONCORRENCIA', 'NCORRENCIA']
    inexigibilidade = ['INEXIGIBILIDADE', 'NEXIGIBILIDADE', 'Inexibilidade', 'XIGIBILIDADE',
     'xigibilidade', 'EXIGIBILIDADE', 'exigibilidade']
    tomada_de_preco = ['TOMADA DE PRECO', 'OMADA DE PRECO', 'omada de preco']

    if (type(proc) != str):
        return None
    try:
        if (proc in isento):
            return 'isento'
        elif (proc in pregao):
            return 'pregao'
        elif (proc in dispensa):
            return 'dispensa'
        elif (proc in concorrencia):
            return 'concorrencia'
        elif (proc in inexigibilidade):
            return 'inexigibilidade'
        elif (proc in tomada_de_preco):
            return 'tomada de preco'
        else:
            return proc.lower()
    except TypeError:
        pass

pagamentos['processo_licitatorio'] = pagamentos['processo_licitatorio'].apply(limpar_proc_licitatorio)

In [87]:
# Os que da pra conferir no olho:
# funcao, subfuncao, fonte_recurso
def limpar_natureza(natureza):
    combustivel_lubrificante = ['combustiveis e lub automotivos', 'combustiveis e lubrificantes', 'combust']
    confeccao_fardamento = ['confeccao de fardamento', 'confeccao de fardamentos']

    if (natureza in combustivel_lubrificante):
        return 'combustiveis e lubrificantes'
    elif (natureza in confeccao_fardamento):
        return 'confeccao de fardamento'


In [88]:
def clean_str(text):
    if (type(text) == float):
        return ''

    # Remove pontuacao, digitos e espacos
    text = ' '.join(re.findall(r'\b[a-zÀ-ú]+\b', text.lower()))
    
    # Remove acentos, cedilhas etc
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = ''.join([char for char in nfkd_form if not unicodedata.combining(char)])

    # Remove stopwords
    my_stopwords = stopwords.words('portuguese')
    my_words = ['desta', 'deste', 'qualquer', 'das']
    my_stopwords = my_stopwords + my_words

    return ' '.join([word for word in text.split() if word not in my_stopwords])

In [89]:
# Removendo pontuacao e numeros de campos com string
# TODO: Tentar mudar o pipeline de stopwords
# pra utilizar stopwords em portugues

pagamentos['credor'] = pagamentos['credor'].apply(clean_str)
pagamentos['bem_ou_servico_prestado'] = pagamentos['bem_ou_servico_prestado'].apply(clean_str)
pagamentos['natureza'] = pagamentos['natureza'].apply(clean_str)
pagamentos['acao'] = pagamentos['acao'].apply(clean_str)
pagamentos['funcao'] = pagamentos['funcao'].apply(clean_str)
pagamentos['subfuncao'] = pagamentos['subfuncao'].apply(clean_str)
pagamentos['fonte_recurso'] = pagamentos['fonte_recurso'].apply(clean_str)

# pagamentos.nunique()

In [120]:
print(pagamentos.info())
pagamentos.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160211 entries, 13 to 427803
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   credor                   160211 non-null  object 
 1   valor                    160211 non-null  float64
 2   bem_ou_servico_prestado  160211 non-null  object 
 3   natureza                 160211 non-null  object 
 4   acao                     160211 non-null  object 
 5   funcao                   160211 non-null  object 
 6   subfuncao                160211 non-null  object 
 7   processo_licitatorio     141472 non-null  object 
 8   fonte_recurso            160211 non-null  object 
dtypes: float64(1), object(8)
memory usage: 17.2+ MB
None


,credor,valor,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
13,aurimar gomes machado,30.00,diaria ipecaeta,diarias,manutencao ipfs,previdencia social,administracao geral,isento,cont regime prpprio prev socia
14,ediney cardoso gomes,30.00,diaria ipecaeta,diarias,manutencao ipfs,previdencia social,administracao geral,isento,cont regime prpprio prev socia
15,cristiane silva villa flor,30.00,diaria ipecaeta,diarias,manutencao ipfs,previdencia social,administracao geral,isento,cont regime prpprio prev socia
18,telemar telecomun bahia sa,1213.20,despesa telefone fixo smt,servicos telecomunicacoes,manutencao smt,administracao,administracao geral,isento,rec proprias ent adm
190,h t hosp t ortopedia lt,101664.16,ref servicos medico prestados pacientes sus,serv medico hospitalar odont lab,atencao media alta complexibilidade amb hospit...,saude,assistencia hospitalar ambul,convite,transf rec sus


## Extraindo features
Verificar:
- TF-IDF
- TSNE

## Prevendo o valor do pagamento

In [96]:
pagamentos.head()
# pagamentos.nunique()



,credor,valor,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
13,aurimar gomes machado,30.00,diaria ipecaeta,diarias,manutencao ipfs,previdencia social,administracao geral,isento,cont regime prpprio prev socia
14,ediney cardoso gomes,30.00,diaria ipecaeta,diarias,manutencao ipfs,previdencia social,administracao geral,isento,cont regime prpprio prev socia
15,cristiane silva villa flor,30.00,diaria ipecaeta,diarias,manutencao ipfs,previdencia social,administracao geral,isento,cont regime prpprio prev socia
18,telemar telecomun bahia sa,1213.20,despesa telefone fixo smt,servicos telecomunicacoes,manutencao smt,administracao,administracao geral,isento,rec proprias ent adm
190,h t hosp t ortopedia lt,101664.16,ref servicos medico prestados pacientes sus,serv medico hospitalar odont lab,atencao media alta complexibilidade amb hospit...,saude,assistencia hospitalar ambul,convite,transf rec sus
